# Intro to Recurrent Neural Networks
### Solution Code
* **PyData Bristol - 5th Meetup:** https://www.meetup.com/PyData-Bristol/events/255667468/
* **Event URL:** https://www.eventbrite.co.uk/e/intro-to-recurrent-neural-networks-tickets-52401888459
* **Date:** Tue 13th November 2018
* **Instructor:** John Sandall
* **Contact:** john@coefficient.ai / @john_sandall

---

In [38]:
import numpy as np

%matplotlib inline
np.random.seed(0)

## Lab: Build A Recurrent Neural Network

Let's build a basic RNN using just numpy. We won't train it for now, we'll instead just get a feeling for how it's working. We'll use input data that has 20 samples, each with two-features, and two time points (t=0 and t=1).

In [39]:
n_features = 2
n_samples = 20

In [40]:
# Create our input data. Here's X at t=0
X0 = np.random.randint(low=-10, high=10, size=(n_samples, n_features))
X0

array([[  2,   5],
       [-10,  -7],
       [ -7,  -3],
       [ -1,   9],
       [  8,  -6],
       [ -4,   2],
       [ -9,  -4],
       [ -3,   4],
       [  7,  -5],
       [  3,  -2],
       [ -1,   9],
       [  6,   9],
       [ -5,   5],
       [  5, -10],
       [  8,  -7],
       [  7,   9],
       [  9,   9],
       [  4,  -3],
       [-10,  -9],
       [ -1, -10]])

In [41]:
# Similarly here's X at t=1
X1 = np.random.randint(low=-10, high=10, size=(n_samples, n_features))

Let's also create the weight matrices `Wx` (connecting X to neurons) and `Wy` (connecting output y at t-1 to neurons at time t).

In [42]:
n_neurons = 3

# Connects 2-features to 3-neurons
Wx = np.random.randint(low=-5, high=5, size=(n_features, n_neurons))
Wx

array([[-1, -2,  2],
       [ 0,  0, -5]])

In [43]:
# Connects 3-neuron output at time t-1 to 3-neurons at time t (the recurrent weights)
Wy = np.random.randint(low=-5, high=5, size=(n_neurons, n_neurons))
Wy

array([[-4,  0,  4],
       [-2, -5,  0],
       [-5, -4, -3]])

In [44]:
# We'll also need the bias
b = np.ones(n_neurons)
b

array([1., 1., 1.])

> #### Exercise: Calculate Y0!
> 
> **Tips**:
> - Remember `Y0 = activation(X0*Wx + b)` and `Y1 = activation(X0*Wx + Y0*Wy + b)`
> - You'll need `np.matmul()` to do multiply two matrixes.
> - You'll need `np.heaviside(some_vector, 0)` for your activation function.

In [45]:
# How does matmul work?
np.matmul(X0, Wx)

array([[ -2,  -4, -21],
       [ 10,  20,  15],
       [  7,  14,   1],
       [  1,   2, -47],
       [ -8, -16,  46],
       [  4,   8, -18],
       [  9,  18,   2],
       [  3,   6, -26],
       [ -7, -14,  39],
       [ -3,  -6,  16],
       [  1,   2, -47],
       [ -6, -12, -33],
       [  5,  10, -35],
       [ -5, -10,  60],
       [ -8, -16,  51],
       [ -7, -14, -31],
       [ -9, -18, -27],
       [ -4,  -8,  23],
       [ 10,  20,  25],
       [  1,   2,  48]])

In [46]:
# How does heaviside work?
np.heaviside([1, -1, 10], 0)

array([1., 0., 1.])

In [47]:
# Calculate Y0 = activation(X0*Wx + b) where activation is heaviside function
Y0 = np.heaviside(np.matmul(X0, Wx) + b, 0)
Y0

array([[0., 0., 0.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 0.],
       [0., 0., 1.],
       [1., 1., 0.],
       [1., 1., 1.],
       [1., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 1., 0.],
       [0., 0., 0.],
       [1., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 1.],
       [1., 1., 1.],
       [1., 1., 1.]])

In [48]:
# Calculate Y1 = activation(X0*Wx + Y0*Wy + b)
Y1 = np.heaviside(np.matmul(X1, Wx) + np.matmul(Y0, Wy) + b, 0)
Y1

array([[1., 1., 1.],
       [0., 0., 0.],
       [0., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [0., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 1., 1.],
       [0., 1., 0.],
       [1., 1., 1.],
       [0., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 1., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 0.]])

## Lab: Build A Recurrent Neural Network using Keras

Let's work through a simple example now using Keras.

In [49]:
from keras.layers import SimpleRNN, Dense, TimeDistributed
from keras.models import Sequential

In [50]:
# Check if Keras is using GPU version of TensorFlow
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9166241119897375039
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7439476327
locality {
  bus_id: 1
  links {
  }
}
incarnation: 5443366956250786997
physical_device_desc: "device: 0, name: Tesla M60, pci bus id: 0000:00:1b.0, compute capability: 5.2"
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 7439538586
locality {
  bus_id: 1
  links {
  }
}
incarnation: 3995675864391442303
physical_device_desc: "device: 1, name: Tesla M60, pci bus id: 0000:00:1c.0, compute capability: 5.2"
, name: "/device:GPU:2"
device_type: "GPU"
memory_limit: 7439538586
locality {
  bus_id: 1
  links {
  }
}
incarnation: 2254771597575135775
physical_device_desc: "device: 2, name: Tesla M60, pci bus id: 0000:00:1d.0, compute capability: 5.2"
, name: "/device:GPU:3"
device_type: "GPU"
memory_limit: 7439538586
locality {
  bus_id: 1
  links {
  }
}
incarnation: 17692974588325667398
physical_device_desc:

Let's now look at 5 time steps, with:
- input X has 20 samples and two features
- output y is of length 3 (we have three neurons).

In [51]:
# Input format shape for Keras is (sample size, number of time steps, features)
n_steps = 5

X = np.random.randint(low=-10, high=10, size=(n_samples, n_steps, n_features))
X.shape

(20, 5, 2)

In [52]:
y = np.random.randint(low=-10, high=10, size=(n_samples, n_steps, n_neurons))
y.shape

(20, 5, 3)

> #### Exercise: Define a simple `Sequential` RNN model using Keras
> - The model should contain one layer (`SimpleRNN` with 3 units, and `return_sequences=True`
> - Assign it to a variable called `model`
> - Use the Keras documentation if you get stuck!

In [53]:
# Define
model = Sequential([
    SimpleRNN(units=3, return_sequences=True),
])

> #### Exercise: Compile & fit the model
> - Use MSE loss and `rmsprop` optimizer.
> - Fit it to X and y, using 10 epochs and batch size of 32.

In [54]:
model.compile(loss="mse", optimizer="rmsprop")

In [55]:
model.fit(X, y, epochs=10, batch_size=32)

Epoch 1/10
20/20 [==============================] - 1s 33ms/step - loss: 35.1246
Epoch 2/10
20/20 [==============================] - 0s 387us/step - loss: 35.1137
Epoch 3/10
20/20 [==============================] - 0s 371us/step - loss: 35.1062
Epoch 4/10
20/20 [==============================] - 0s 377us/step - loss: 35.1002
Epoch 5/10
20/20 [==============================] - 0s 377us/step - loss: 35.0950
Epoch 6/10
20/20 [==============================] - 0s 368us/step - loss: 35.0904
Epoch 7/10
20/20 [==============================] - 0s 352us/step - loss: 35.0862
Epoch 8/10
20/20 [==============================] - 0s 350us/step - loss: 35.0823
Epoch 9/10
20/20 [==============================] - 0s 356us/step - loss: 35.0787
Epoch 10/10
20/20 [==============================] - 0s 361us/step - loss: 35.0753


Let's try it out! We'll generate some new data `X_new` in the same shape as X.

In [56]:
# We'll have one sample, so we want it to have shape (1, 5, 2)
X.shape

(20, 5, 2)

In [57]:
# This has shape (1, 5, 2)
X_new = np.array([
    [[1, 0],  # t = 0 (two features)
     [0, 1],  # t = 1
     [0, 1],  # t = 2
     [0, 1],  # t = 3
     [0, 1],  # t = 4
    ]
])
X_new.shape

(1, 5, 2)

In [58]:
# Our RNN is able to predict some outcomes y of length 3, for each time step.
model.predict(X_new)

array([[[ 0.65909064,  0.23564485,  0.79182005],
        [-0.06538551,  0.7689222 ,  0.5821736 ],
        [-0.7504765 ,  0.6854172 ,  0.57146424],
        [-0.8769358 ,  0.3338538 ,  0.38439777],
        [-0.85083365,  0.24807689,  0.02401483]]], dtype=float32)

> #### Exercise: Predict single value outputs for y (instead of vectors of length 3)
> - Within your `Sequential` model, add a fully connected `Dense()` network with `input_dim=1` and `output_dim=1`
> - Compile as before
> - Fit to the new y provided
> - Predict for `X_new` again, confirming that your outputs are a single time series of 5 numbers.

In [59]:
# We want a newly shaped y to predict, containing 20 samples over 5 time steps, but otherwise scalar output.
y = np.random.randint(low=-10, high=10, size=(n_samples, n_steps, 1))
y.shape

(20, 5, 1)

In [60]:
# Define
model = Sequential([
    SimpleRNN(units=3, return_sequences=True),
    Dense(input_dim=3, units=1),
])

model.compile(loss='mse', optimizer='rmsprop')
model.fit(X, y, epochs=10, batch_size=32)

Epoch 1/10
20/20 [==============================] - 1s 39ms/step - loss: 41.4505
Epoch 2/10
20/20 [==============================] - 0s 383us/step - loss: 41.3444
Epoch 3/10
20/20 [==============================] - 0s 408us/step - loss: 41.2679
Epoch 4/10
20/20 [==============================] - 0s 403us/step - loss: 41.2040
Epoch 5/10
20/20 [==============================] - 0s 379us/step - loss: 41.1475
Epoch 6/10
20/20 [==============================] - 0s 378us/step - loss: 41.0959
Epoch 7/10
20/20 [==============================] - 0s 401us/step - loss: 41.0477
Epoch 8/10
20/20 [==============================] - 0s 371us/step - loss: 41.0021
Epoch 9/10
20/20 [==============================] - 0s 397us/step - loss: 40.9585
Epoch 10/10
20/20 [==============================] - 0s 404us/step - loss: 40.9164


In [61]:
model.predict(X_new)

array([[[-1.8910395 ],
        [ 0.05946516],
        [ 0.13487016],
        [-0.87448853],
        [-1.0239842 ]]], dtype=float32)

> #### Exercise: Train a more fully fledged RNN on real data.
> - We'll construct an X input with `1` at t=0 and `0` otherwise.
> - Our `y` output just has a simple pattern.
> - The RNN should be able to learn the relationship between the X pattern, and the corresponding y pattern.
> - Re-use your code from before, i.e. a Sequential model containing a SimpleRNN (this time with 50 units), plus a Dense layer with 1 unit and `sigmoid` activation.
> - Compile as before, and fit to `x_train` and `y_train` using 10 epochs.

In [62]:
# These are our sequences. The RNN should learn to predict the
# 0.8 and 0.6 correctly because it can remember the 1 in the inputs.
x_seed = [1, 0, 0, 0, 0, 0]
y_seed = [1, 0.8, 0.6, 0, 0, 0]

In [63]:
# Let's create 1000 identical samples.
n_samples = 1000

x_train = np.array([[x_seed] * n_samples]).reshape(n_samples, len(x_seed), 1)
y_train = np.array([[y_seed] * n_samples]).reshape(n_samples, len(y_seed), 1)

x_train.shape

(1000, 6, 1)

In [64]:
# Define your model here...
model = Sequential([
    SimpleRNN(units=50, return_sequences=True),
    Dense(units=1, activation="sigmoid"),
])

In [65]:
# Compile...
model.compile(loss="mse", optimizer="rmsprop")

In [66]:
# Fit...
model.fit(x_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
1000/1000 [==============================] - 1s 972us/step - loss: 0.0627
Epoch 2/10
1000/1000 [==============================] - 0s 254us/step - loss: 0.0151
Epoch 3/10
1000/1000 [==============================] - 0s 256us/step - loss: 0.0055
Epoch 4/10
1000/1000 [==============================] - 0s 253us/step - loss: 0.0022
Epoch 5/10
1000/1000 [==============================] - 0s 265us/step - loss: 0.0011
Epoch 6/10
1000/1000 [==============================] - 0s 272us/step - loss: 7.5417e-04
Epoch 7/10
1000/1000 [==============================] - 0s 265us/step - loss: 5.7742e-04
Epoch 8/10
1000/1000 [==============================] - 0s 247us/step - loss: 4.8171e-04
Epoch 9/10
1000/1000 [==============================] - 0s 253us/step - loss: 4.2233e-04
Epoch 10/10
1000/1000 [==============================] - 0s 257us/step - loss: 3.8175e-04


In [67]:
# Let's predict for this x_new
x_new = np.array([[[1],[0],[0],[0],[0],[0]]])
x_new

array([[[1],
        [0],
        [0],
        [0],
        [0],
        [0]]])

In [68]:
model.predict(x_new)

array([[[0.972178  ],
        [0.79625285],
        [0.5646119 ],
        [0.00548737],
        [0.00533291],
        [0.00398993]]], dtype=float32)

In [69]:
# Calculate Mean Squared Error.
def mse(predictions):
    return np.square(np.array(y_seed) - predictions[0].reshape(-1)).mean()

print("\nMSE = ", mse(model.predict(x_new)))


MSE =  0.00035248220503706257


In [70]:
# Let's try train for longer and use adam optimizer.
# Does it get better at matching the [1, 0.8, 0.6, 0, 0, 0] pattern?
model.compile(loss="mse", optimizer="adam")
model.fit(x_train, y_train, epochs=100, batch_size=32, verbose=0)
print(model.predict(x_new))
print("\nMSE = ", mse(model.predict(x_new)))

[[[9.9835593e-01]
  [8.0000287e-01]
  [6.0001141e-01]
  [4.7703512e-04]
  [4.2098717e-04]
  [2.8200637e-04]]]

MSE =  5.312401720191284e-07


## Lab: LSTMs and GRUs

In [71]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM, GRU

> #### Exercise: Try using the LSTM and GRU units from Keras on the previous example. Does it appear to perform any better?

In [72]:
# Let's keep everything else the same for the LSTM
model = Sequential([
    LSTM(units=50, return_sequences=True),
    Dense(units=1, activation="sigmoid"),
])
model.compile(loss="mse", optimizer="adam")
model.fit(x_train, y_train, epochs=100, batch_size=32, verbose=0)
print(model.predict(x_new))
print("\nMSE = ", mse(model.predict(x_new)))

[[[9.9241841e-01]
  [7.9999602e-01]
  [6.0000068e-01]
  [4.1481841e-04]
  [1.9005934e-07]
  [5.1780169e-08]]]

MSE =  9.608770324705262e-06


In [73]:
# Similarly for the GRU
model = Sequential([
    GRU(units=50, return_sequences=True),
    Dense(units=1, activation="sigmoid"),
])
model.compile(loss="mse", optimizer="adam")
model.fit(x_train, y_train, epochs=100, batch_size=32, verbose=0)
print(model.predict(x_new))
print("\nMSE = ", mse(model.predict(x_new)))

[[[9.9287969e-01]
  [7.9989284e-01]
  [5.9936571e-01]
  [2.7148754e-04]
  [2.3575201e-05]
  [1.9768560e-05]]]

MSE =  8.531214908966155e-06


> #### Exercise: Try adding some additional components from the example provided [on the Keras docs here](https://keras.io/getting-started/sequential-model-guide/), such as Dropout. How does this improve things?

In [74]:
# Add dropout
model = Sequential([
    GRU(units=50, return_sequences=True),
    Dropout(0.5),
    Dense(units=1, activation="sigmoid"),
])
model.compile(loss="mse", optimizer="adam", metrics=['accuracy'])
model.fit(x_train, y_train, epochs=100, batch_size=32)
print(model.predict(x_new))
print("\nMSE = ", mse(model.predict(x_new)))

Epoch 1/100
1000/1000 [==============================] - 2s 2ms/step - loss: 0.1583 - acc: 0.6410
Epoch 2/100
1000/1000 [==============================] - 1s 512us/step - loss: 0.0916 - acc: 0.6653
Epoch 3/100
1000/1000 [==============================] - 0s 498us/step - loss: 0.0422 - acc: 0.6667
Epoch 4/100
1000/1000 [==============================] - 1s 502us/step - loss: 0.0165 - acc: 0.6667
Epoch 5/100
1000/1000 [==============================] - 1s 507us/step - loss: 0.0091 - acc: 0.6667
Epoch 6/100
1000/1000 [==============================] - 0s 499us/step - loss: 0.0074 - acc: 0.6667
Epoch 7/100
1000/1000 [==============================] - 0s 497us/step - loss: 0.0064 - acc: 0.6667
Epoch 8/100
1000/1000 [==============================] - 1s 503us/step - loss: 0.0055 - acc: 0.6667
Epoch 9/100
1000/1000 [==============================] - 1s 522us/step - loss: 0.0050 - acc: 0.6667
Epoch 10/100
1000/1000 [==============================] - 1s 505us/step - loss: 0.0045 - acc: 0.6667
E

1000/1000 [==============================] - 1s 514us/step - loss: 1.8336e-04 - acc: 0.6667
Epoch 81/100
1000/1000 [==============================] - 1s 508us/step - loss: 1.8323e-04 - acc: 0.6667
Epoch 82/100
1000/1000 [==============================] - 1s 520us/step - loss: 1.8687e-04 - acc: 0.6667
Epoch 83/100
1000/1000 [==============================] - 1s 503us/step - loss: 1.7981e-04 - acc: 0.6667
Epoch 84/100
1000/1000 [==============================] - 1s 518us/step - loss: 1.8153e-04 - acc: 0.6667
Epoch 85/100
1000/1000 [==============================] - 1s 513us/step - loss: 1.8124e-04 - acc: 0.6667
Epoch 86/100
1000/1000 [==============================] - 1s 525us/step - loss: 1.7469e-04 - acc: 0.6667
Epoch 87/100
1000/1000 [==============================] - 1s 530us/step - loss: 1.7234e-04 - acc: 0.6667
Epoch 88/100
1000/1000 [==============================] - 1s 524us/step - loss: 1.5817e-04 - acc: 0.6667
Epoch 89/100
1000/1000 [==============================] - 1s 515us/s

> #### Suggested "homework" exercise: Work through the Keras "text generation example" code: https://github.com/keras-team/keras/blob/master/examples/lstm_text_generation.py
> 
> Try applying this to your own text dataset!

In [75]:
# Required imports for this example
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

> The example below is inspired by [this gist](https://gist.github.com/nylki/1efbaa36635956d35bcc). The recipes are from [ffts.com](http://www.ffts.com/recipes.htm). If the URL below stops working, [here is the archive.org copy](http://web.archive.org/web/20170619161312/http://www.ffts.com/recipes/lg/lg32965.zip). The ZIP is ~12MB compressed, 39MB uncompressed, and contains various files in plaintext but in a format intended to be read by a program called Meal-Master, hence the start/end lines.

In [91]:
# Grab some fun recipe data
import requests, zipfile, io
from pathlib import Path

r = requests.get("http://www.ffts.com/recipes/lg/lg32965.zip")
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall(path="recipes")

In [106]:
recipe_files = []
recipe_files.sort()
text = ""
for f in Path('./recipes').glob('*.mmf'):
    with io.open('./recipes/' + f.name, encoding='latin1') as f:
        text += f.read().lower()

# Let's have a peek
print(text[:2000])

mmmmm----- recipe via meal-master (tm) v8.05
 
      title: crunchy canyon snack mix *
 categories: snacks
      yield: 8 cups
 
mmmmm-------------------------bills20086------------------------------
      3 c  mini pretzel twists
  2 1/2 c  wedge shaped cheese crackers
      2 c  mini garlic bagel chips;
           - =or= mini garlic rye toast
  3 1/4 oz italian breadsticks; break
           - into thirds
    1/3 c  butter or margarine; melted
      1 tb worcestershire sauce
      1 ts lemon juice
    1/2 ts garlic powder
    1/2 ts seasoned salt
    1/4 ts cayenne pepper
 
  heat oven to 350~. in large bowl, combine pretzel twists, crackers,
  bagel chips and breadsticks. in small bowl, combine remaining
  ingredients; mix well. drizzle margarine mix over pretzel mix; toss
  to coat. spread evenly in ungreased jelly-roll pan. bake at 350~ for
  10-15 minutes or until crisp, stirring occasionally. cool completely.
  store in tightly covered container.
 
mmmmm
 
mmmmm----- recipe via m

In [107]:
# Load the dataset from a file
print('corpus length:', len(text))

corpus length: 38110009


In [111]:
# The example recommends 100k to 1MM characters, so 38MM is probably too much (for now)!
text = text[:100000]
print('corpus length:', len(text))

corpus length: 100000


In [112]:
# How many unique characters?
chars = sorted(list(set(text)))
print('total chars:', len(chars))

# Create a lookup dictionary so we can go from characters to integers
char_indices = dict((c, i) for i, c in enumerate(chars))
char_indices

# Create another lookup dictionary so we can go from integers back to characters
indices_char = dict((i, c) for i, c in enumerate(chars))
indices_char

total chars: 69


{0: '\t',
 1: '\n',
 2: ' ',
 3: '!',
 4: '"',
 5: '$',
 6: '%',
 7: '&',
 8: "'",
 9: '(',
 10: ')',
 11: '*',
 12: '+',
 13: ',',
 14: '-',
 15: '.',
 16: '/',
 17: '0',
 18: '1',
 19: '2',
 20: '3',
 21: '4',
 22: '5',
 23: '6',
 24: '7',
 25: '8',
 26: '9',
 27: ':',
 28: ';',
 29: '<',
 30: '=',
 31: '>',
 32: '?',
 33: '@',
 34: '[',
 35: ']',
 36: '_',
 37: 'a',
 38: 'b',
 39: 'c',
 40: 'd',
 41: 'e',
 42: 'f',
 43: 'g',
 44: 'h',
 45: 'i',
 46: 'j',
 47: 'k',
 48: 'l',
 49: 'm',
 50: 'n',
 51: 'o',
 52: 'p',
 53: 'q',
 54: 'r',
 55: 's',
 56: 't',
 57: 'u',
 58: 'v',
 59: 'w',
 60: 'x',
 61: 'y',
 62: 'z',
 63: '{',
 64: '}',
 65: '~',
 66: '®',
 67: 'á',
 68: 'ø'}

In [113]:
# Cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 33320


In [114]:
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


In [115]:
# Build the model: a single LSTM
print('Build model...')
model = Sequential([
    LSTM(128, input_shape=(maxlen, len(chars))),
    Dense(len(chars), activation='softmax'),
])
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [116]:
def sample(preds, temperature=1.0):
    """Helper function to sample an index from a probability array."""
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, _):
    """Function invoked at end of each epoch. Prints generated text."""
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [117]:
# Callback executes the on_epoch_end function to generate some text at the end of each epoch.
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

#### Let's generate some text!

From the [example this code is taken from](https://github.com/keras-team/keras/blob/master/examples/lstm_text_generation.py):

> At least 20 epochs are required before the generated text starts sounding coherent. It is recommended to run this script on GPU, as recurrent networks are quite computationally intensive. If you try this script on new data, make sure your corpus has at least ~100k characters. ~1M is better.

**Note:** I've run the following on an AWS g3.16xlarge instance. It takes 23 seconds per epoch.

In [118]:
model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback]
)

Epoch 1/60
33320/33320 [==============================] - 23s 680us/step - loss: 2.4490

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "r
    1/4 ts salt
    1/3 c  butter
    "
r
    1/4 ts salt
    1/3 c  butter
      1/4 ts crunthe cheated cringer and ther and ther and cherpertester chepper and crucherserates ther and crunthester and preated crutted creaserested cheaterested creates
  
      1/4 ts and ther and createster and ther and ther and creshertes ther chither chinger and ther ther thes and crucherserserester and minutes
      1/4 ts corked chiperester cherted preatesterted creathed preatester and cre
----- diversity: 0.5
----- Generating with seed: "r
    1/4 ts salt
    1/3 c  butter
    "
r
    1/4 ts salt
    1/3 c  butter
      28 ts baltty the sirmen thestester ther ander baterastalt
      1/4 ts partes and ther thintalis angartesters
      1/4 ts trestaltesticherred watter perked crich angarthestingroserongartesters ingrecesteron

  easlly connare ckned in coated melesting doney, lish thow  popp in margarine mintires.wgestast-----------------------------------------------------------------------------------l------------------------------------- in larg--------------
  
  hop froz the
  to corn flake dremorill the pold-capes in fthe
----- diversity: 1.2
----- Generating with seed: " oz spinach, frozen cut; thawed
      8 "
 oz spinach, frozen cut; thawed
      8 c  sucad quicht brasinf & virets--, carrings, amo0-umouscan-an for 3.2 ftpjrater the
 .brishs0 thated blennt yo2 28 t9 minutes,  turd 1
  20 minutes. crincal, pinetarm wapgin serve ir fnonntbz
  powate ingar dook toth---die fat eggs, bread cremse eill 1 bush. ad and colkd. add-shack co pcated hoth
  nntboon salating skilien in dish dirinm.
 
mmmmm
 
mmmmm----- recipe via meal-master (tm) v8.05
 
 
Epoch 5/60
33320/33320 [==============================] - 22s 664us/step - loss: 1.2451

----- Generating text after Epoch: 4
----- diversity: 0.2
----- Generat

---------------gingersnap-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
----- diversity: 0.5
----- Generating with seed: "---------------gingersnap---------------"
---------------gingersnap--------------------
    1/4 c  serves ar
      1 ts vegetable, chocolate on wienntred are serve with molkes, stir in loast to pour the depientore this a ling for a bottor and shary extrass and stor corn breads and sauce
      3 tb brown cheese
           inanges
      2 ts beat eggs
    1/4 ts bake ois
      3 c  pean on papted with parsled
    1/4 ts bake with
      3 tb top breads
      1 ts baking powder
   
----- diversity: 1.0
-----

      titted crushen white
      3 cn cristborbsef hoght 1
           -ol sproiced
      1 tb water
 
----- diversity: 1.2
----- Generating with seed: " may-june 1994 typed for
  you by linda "
 may-june 1994 typed for
  you by linda oi 5 t: tbleppoug
 
mmmmm
 
mmmmm----- recipe via meal-master (tm) v8.05
 
      titied vegetable .p
  to meat, clave 44 mecomband -give. tomathid half of
  greased 4. pir tombineontil by minuts
 
mmmmm
 
mmmmm----- recipe via meal-master (tm) v8.05
 
      titted oniclow,
  1/2utyp flovem 1, sugar.
  
  porkig sinn134 suts or th ---fitfor 199p blings on owt at
   dyfruhhevefon
  piess. to sealy d
Epoch 12/60
33320/33320 [==============================] - 22s 658us/step - loss: 0.8672

----- Generating text after Epoch: 11
----- diversity: 0.2
----- Generating with seed: "ervings
 
    1/2 c  (8 tbsp) honey
    "
ervings
 
    1/2 c  (8 tbsp) honey
      1 c  ald:                                                                                              

           stoces
    1/2 c  salt
           -- trais
      1 c  crastables; fresh onions, until green onions, chips until fing to a wegteral combine corn chillis as in a 3 hourd yeast in a serve with or with the chillis, 1/2 to can greaser. surfack beft eggs oves the chicken salt, covered sided)
  
  tomatoes: dip with butter and stir in of the into the salt. bake and cream. lestry cover with lead in a bookhe
----- diversity: 1.0
----- Generating with seed: "
      yield: 6 servings
 
           st"

      yield: 6 servings
 
           stirm-thanhine brown suefr
 
mmmmm
 
mmmmmmm--------- 3---------------m---------- 19-epg,  milk, chopped.weft wind theast with chevolest fresh to cornf. sound in cinnit ingredients. combine
 
mmmmm
 
mmmmmmm---sit-mice ce
 mitt, ttive. ate35v.  dough to 300 deg fomea bumb sual a one quant lightly. repure fer 1hy ty food to foot recipe bageled for at uscremastuns or over is stir gelecr celtinues, fra
----- diversity: 1.2
----- Generating with seed: "
  

...

r 15 to 20 minutes. flour a board with the on firm knite crushed tomatoes-as (ing for 5 thing with notsed weighte of brush to beat fillet into bowly. let the insers.
  
  cooking reduie bacon. cut into 115m for 5 minute of the each bean ce5ter in corninuee, wanchy. lead cane or boil; hazelnut recipes and
  cracker willet or tomatoes -- it if calories, 1/2 fat, 1/4 vegetables
      yield: 4 servings
 
    1/4 c  onion, milk
      1 ts gr
----- diversity: 1.0
----- Generating with seed: "r 15 to 20 minutes. flour a board with t"
r 15 to 20 minutes. flour a board with the table bould pan.  prehoat oven top. pepper. red chillis milk,ne salt
 
  1 servenis all the ecga, minill until crushed combine honds of puafking. cover
 
mmmmm
 
mmmmm----- recipe via meal-master (tm) v8.0
 
           tim into chicken nuect
    1/4 c  sermosin;
     10 oz spreased worenuts
      1 ts baking knite and leggs
    1/2 ts salt
 
  in large sovinch baye pie panct youls, stirring not
----- diversity: 1.2
-----

  firs) recim and a sine, halfan sifted flour and raising with cooking sue-d)
  
  roll olive 4 tilly 
 
mmmmm
 
m
Epoch 49/60
33320/33320 [==============================] - 22s 665us/step - loss: 0.4935

----- Generating text after Epoch: 48
----- diversity: 0.2
----- Generating with seed: "r an out-of-doors treat, car ride (donna"
r an out-of-doors treat, car ride (donnatures, elice
  fish typed of the into the top wich broholbor the dough into 1/4 ingredients in a shredientsing. melts, suive in highty freser for ato recife seal of tugely cheese and continue
  thr4y basil
  source: 1 1/2 eags
           -- mine
           -- finely chopped
      1 ts seer
      2 tb blan crushed corn olive, peeletoons the mare and green eggsard whith brown sugar, continue
  chees
----- diversity: 0.5
----- Generating with seed: "r an out-of-doors treat, car ride (donna"
r an out-of-doors treat, car ride (donnat and spread into 1/2 cup of tugely cheese, sauce
           -- stir and lek-
           -pi

           white tuna packed and
      yield: 4 servings
 
      1 c  rolleds
    1/4 c  hor cream
           -mine sactick
      3 oz fresh or
      3 tb semins sauce 
  semalor
    1/2 c  brown sugar, bakly sauce
      2 lb preparo, slices, baniss, ard faver chen and salt and peanut weings
  using all ule by swith tham coulde cover isingart
  baking sheet with a lageter, combine,
  baking sheets. add to pie pan. place you d
----- diversity: 1.2
----- Generating with seed: " can solid
           white tuna packed "
 can solid
           white tuna packed packed breat the
m mgg red pepper miln mestsi all
  microwing. 
  to basils. brown suraoulmer 14
  5 garmot cream uptroco seciame beat oven 2. info maver 1-doug. stices evenly 3/4
 
  60omander and saese passe comenom.
  
  preesabe until fings.
           -out
      yield: 6 servings
 
      yield: 2 ostery ric of brown sauce 3(out the or with smooltycr.
  pankate 4.2..s
 
mmmmm
 
mmmmm----- reci
Epoch 53/60
33320/33320 [============

    1/4 c  skit themes
      1 cn hour or mach fine, thed crushed cornflakes, saltite on paices, tomature
  altires. sateat basils of skity until breads, sugar.
  
  remove from the 200 f.                                      
----- diversity: 0.5
----- Generating with seed: "   flour, all-purpose
      1    egg; be"
   flour, all-purpose
      1    egg; beet pepper
    1/4 ts ground mill
    1/4 c  onionchops; 1/2 fat, 1/4 frumb
           deprose dip
           drained crighen sauce
    1/4 ts salt
    1/4 ts varile ofiend dars
    1/4 ts onions, moontle graes or with whents to a bowl, crushed topk. set. they haftide. rool pers and cream cheese, sauce and sliced baking the pan. pater cheese, sauce and creaming whire sugar
  
  eachoved tomatoes in
----- diversity: 1.0
----- Generating with seed: "   flour, all-purpose
      1    egg; be"
   flour, all-purpose
      1    egg; beet pepper
    1/1 c  water 11 16, 199) stored safrigaranutes on hazes on down into a 9
  with merally bess n

  cut jack cheese into 1/4 cups belb
 
  wtis nould
  or boiling all 10 minutes, targe bowl;
  fist. the
  cemon recipes and blend be from bombleson. bow, combine mixizen browd pan. or the eggarm
 
mmmmm
 
mmmmm----- recipe via meal-master (tm) v8.05
 
      title: crust mixt, much the peant and crodsed meft
  eggar.  using at altir barge bowl;
  flour, saute oni2. bariush ban snoce. be seed and add salt
  1gg rem. babale
Epoch 60/60
33320/33320 [==============================] - 22s 664us/step - loss: 0.4244

----- Generating text after Epoch: 59
----- diversity: 0.2
----- Generating with seed: "      1    green onion, minced
      2 t"
      1    green onion, minced
      2 tb greese chips
           -strayfust
      1 c  limesons
           -or
           -seel
      1                                                                                                                                                                                                                          

In [134]:
def generate_text(diversity, length):
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    print('-------------------\nGenerating with seed: "' +
          sentence + '" \n-------------------\n')
    #sys.stdout.write(generated)

    for i in range(length):
        if i % 100 == 0:
            sys.stdout.write('\n' + str(i) + '/' + str(length))
            sys.stdout.flush()
        if i % 10 == 0:
            sys.stdout.write('.')
            sys.stdout.flush()
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        #sys.stdout.write(next_char)
        #sys.stdout.flush()
    #print()

    return generated

In [135]:
delicious = generate_text(0.5, 5000)

-------------------
Generating with seed: "ories: cyberealm, main dish, low-cal
   " 
-------------------


0/5000..........
100/5000..........
200/5000.........

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:4: RuntimeWarning: divide by zero encountered in log


.
300/5000..........
400/5000..........
500/5000..........
600/5000..........
700/5000..........
800/5000..........
900/5000..........
1000/5000..........
1100/5000..........
1200/5000..........
1300/5000..........
1400/5000..........
1500/5000..........
1600/5000..........
1700/5000..........
1800/5000..........
1900/5000..........
2000/5000..........
2100/5000..........
2200/5000..........
2300/5000..........
2400/5000..........
2500/5000..........
2600/5000..........
2700/5000..........
2800/5000..........
2900/5000..........
3000/5000..........
3100/5000..........
3200/5000..........
3300/5000..........
3400/5000..........
3500/5000..........
3600/5000..........
3700/5000..........
3800/5000..........
3900/5000..........
4000/5000..........
4100/5000..........
4200/5000..........
4300/5000..........
4400/5000..........
4500/5000..........
4600/5000..........
4700/5000..........
4800/5000..........
4900/5000..........

In [136]:
print(delicious)

ories: cyberealm, main dish, low-cal
           -or white
      1 ts semond slices
      1    egg
           -seel
      1 ez cups (sound diuble (cookfry seeds 1 1/2 onch)
  
  1 crust with chocopes.
  
  each serving is tandes, sliced in hol potatoes and
  sugar. in the thire 2
  for bate
  minute to blend
  flour, stir in salt with cream cheese afout 2 hourd. bake in 350ftt(eggars and sliced in a combine flour, bast
           -or
      2 tb vegetables, fish cheese microwan;
  brown. sugar.  bake at 350
  degree ofichely dagingeral pipcop tos 4. caloroes processor.  dough in laver.  diaine hours, thin to the bread cilr crushed sauce. prehout of the a cut into 1 11
  to 1/2 fut butter until
  golden bread
  flour, spoon melted bowl, combine flour, makes 115-11ecchpess - inch pil pepper and mace with cheesuns a spoon mixture 6/20
  part with a well combine all with onions, shallow
  stir in the type sured in snove
  in a brush the surea and broads combine flour,
  cook on uskzes of mix

### Needs some work...!
Sugary chicken onions and crust doubled tomatoes...sounds tasty!